# Imports

In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
%matplotlib inline

df = pd.read_csv("salario.csv")
count = defaultdict(int)

In [2]:
def testing():
    return df.groupby(["pais","estado","cidade","genero","data"], axis=0, level=None, as_index=False).count()

def classEquivalence(a,b,c,d,e):
    return df[(df["pais"] == a) & (df["estado"] == b) & (df["cidade"] == c) & (df["genero"] == d) & (df["data"] == e)]
    

In [3]:
def toGeneralize(genero): 
    if genero == True: return random.choice(df.columns.drop('salario').drop('id'))
    return random.choice(df.columns.drop('salario').drop('id').drop('genero'))

def nextGeneralization(i,j, value):
    if(j == "pais"): 
        if(df.at[i,"estado"] != "*"): return nextGeneralization(i, "estado", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 2): count["lugar",i] = 3
    elif(j == "estado"):
        if(df.at[i,"cidade"] != "*"): return nextGeneralization(i, "cidade", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 1): count["lugar",i] = 2
    elif(j == "cidade"):
        df.at[i, j] = "*"
        if(count["lugar",i] == 0): count["lugar",i] = 1
    elif(j == "genero"):
        df.at[i, j] = "*"
        if(count["genero",i] == 0): count["genero",i] = 1
    elif(j == "data"):
        if(str(value)[0:5] == "**/**"): df.at[i, j] = "**/**/**"
        elif(str(value)[0:2] == "**"): df.at[i, j] = "**/**"  + str((value)[5:8])
        else:
            df.at[i, j] = "**" + str((value)[2:8])
        
        if(count["data",i] < 3): count["data",i] = count["data",i] + 1
    else:
        df.at[i, j] = "erro"
        

In [4]:
def max_hye(col):
    if (col == 'genero'): return 1
    return 3

Na = 3 #n_atributos semi identificadores
def precision():
    summ =  0
    for index, row in df.iterrows():
        for col in df.columns.drop('id').drop('salario'):
                summ += count[col, index]/max_hye(col)

    return 1 - summ/(df.shape[0] * Na)

In [5]:
def nextt(k, genero): #generaliza classe de equivalencia que não satisfaz k-anonimidade
        test = testing()
        test = test[test['salario'] < k]
        
        for index,aux in test.iterrows():
            a = aux['pais']
            b = aux['estado']
            c = aux['cidade']
            d = aux['genero']
            e = aux['data']
        
            
            j = toGeneralize(genero)

            for i in classEquivalence(a,b,c,d,e).index:
                #print(str(i) + " " + str(j))
                nextGeneralization(i, j, aux[j])
            #print("----------------\n\n")
            

            
def nextt2(qtd): #generaliza qtd individuos aleatorios do dataset
    for q in range(qtd):
        i = random.choice(df.index)
        j = toGeneralize(True)
        #print(str(i) + " " + str(j))
        nextGeneralization(i, j, df.loc[i][j])
        #print("----------------\n\n")



def kanonimity(k):
    while testing()[testing()['salario'] < k].shape[0] > 0:
        test = testing()
        test = test[test['salario'] < k]
        qtd_restante = test.shape[0] 
        print(qtd_restante)
        
        #Caso tenha uma classe completamente generalizada que não satisfaz k-anonimidade,
        #devemos, generalizar individuos mesmo que pertencam a alguma classe que satisfaz k-anonimidade.
        #No caso, vamos generalizar 3* a quantidade necessária, de forma aleatória.
        if(qtd_restante == 1 and test['pais'].values[0]=='*' and test['estado'].values[0]=='*' and test['cidade'].values[0]=='*'
        and test['genero'].values[0]=='*' and test['data'].values[0]=='**/**/**'):
            nextt2(3*(k - test.loc[0]['salario']))
        else:
            #Generalizar genero prejudica muito a precisao.
            #Portanto, só será considerado essa possibilidade para as ultimas 50 classes
            if(qtd_restante > 50): 
                nextt(k, False)
            else:
                nextt(k, True)


        

In [6]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(2)
df.to_csv("Salario_K2.csv")

1000
1000
946
748
513
348
245
165
129
90
72
55
43
38
35
33
24
19
17
14
11
10
10
10
7
6
4
3
2
1
1
1
1
1
CPU times: user 21.6 s, sys: 318 ms, total: 21.9 s
Wall time: 21.2 s


In [7]:
precision() #k =2

0.8403333333333345

In [8]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(4)
df.to_csv("Salario_K4.csv")

1000
999
980
840
622
446
303
198
150
109
82
55
39
30
25
21
18
18
18
17
14
13
9
7
7
7
6
5
4
4
4
2
2
2
1
1
CPU times: user 23.6 s, sys: 363 ms, total: 23.9 s
Wall time: 23.1 s


In [9]:
precision() #k=4

0.8097777777777777

In [10]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(8)
df.to_csv("Salario_K8.csv")

1000
1000
972
853
658
489
336
236
158
125
99
73
53
37
33
24
19
18
14
10
6
5
4
3
2
2
2
1
1
CPU times: user 23.9 s, sys: 256 ms, total: 24.2 s
Wall time: 23.5 s


In [11]:
precision() #k=8

0.7802222222222226

In [12]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(16)
df.to_csv("Salario_K16.csv")

1000
998
973
834
665
495
361
273
197
135
108
80
62
40
35
27
24
22
18
18
15
15
13
13
13
10
10
10
8
7
4
4
2
2
2
2
2
2
2
1
1
1
1
1
1
1
CPU times: user 25.5 s, sys: 216 ms, total: 25.8 s
Wall time: 25 s


In [13]:
precision() #k=16

0.7550000000000009

In [14]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(64)

1000
1000
973
849
685
523
405
292
224
165
128
97
77
61
49
41
36
32
26
24
22
16
14
7
6
6
5
5
5
4
3
1
1
1
CPU times: user 26.2 s, sys: 374 ms, total: 26.6 s
Wall time: 25.6 s


In [15]:
precision() #k=64

0.6813333333333351

In [17]:
testing()

,pais,estado,cidade,genero,data,id,salario
0,*,*,*,*,**/**/**,97,97
1,*,*,*,Female,**/**/**,278,278
2,*,*,*,Female,**/**/19,110,110
3,*,*,*,Female,**/**/20,121,121
4,*,*,*,Male,**/**/**,202,202
5,*,*,*,Male,**/**/19,192,192
